In [85]:
import pandas as pd

#------------------------讀取檔案------------------------------------------------
creditcard = pd.read_csv("UCI_Credit_Card.csv", encoding = 'ISO-8859-1')
df = creditcard.copy()

#---------------------資料前置處理-----------------------------------------------
df = df.drop('ID',axis = 1)

df.EDUCATION = df.EDUCATION.map({1:1,2:2,3:3,4:4,5:0,6:0,0:0})#0視為unknown資料
df.MARRIAGE = df.MARRIAGE.map({0:0,1:1,2:2,3:3})#0視為unknown資料

df = pd.get_dummies(df, columns=['SEX'])
df = pd.get_dummies(df, columns=['EDUCATION'])
df = pd.get_dummies(df, columns=['MARRIAGE'])
df = pd.get_dummies(df, columns=['PAY_0'])
df = pd.get_dummies(df, columns=['PAY_2'])
df = pd.get_dummies(df, columns=['PAY_3'])
df = pd.get_dummies(df, columns=['PAY_4'])
df = pd.get_dummies(df, columns=['PAY_5'])
df = pd.get_dummies(df, columns=['PAY_6'])


In [86]:
#---------------------decision tree-----------------------------------------------
x = df.drop('default.payment.next.month', axis=1)
y = df['default.payment.next.month']

from sklearn.datasets import dump_svmlight_file
dump_svmlight_file(x, y, 'svm-output.libsvm')  # where is your y?
from sklearn.datasets import load_svmlight_file


from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics


data  = MLUtils.loadLibSVMFile(sc,"svm-output.libsvm")
(trainingData, testData) = data.randomSplit([0.75, 0.25])
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)

predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)

#zz=labelsAndPredictions.take(30)
#scoreAndLabels = sc.parallelize(zz)

#算出來怪怪ㄉ 修好了！-------------------------
metrics = MulticlassMetrics(labelsAndPredictions)
precision = metrics.precision(label=1)
recall = metrics.recall(label=1)
Accuracy = metrics.accuracy
print("decision tree Accuracy = %s" % Accuracy)
#算出來怪怪ㄉ-------------------------


Accuracy = 0.812269615587


In [87]:
#---------------------RandomForest-----------------------------------------------
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)


#print('Learned classification forest model:')
#print(model.toDebugString())

# Save and load model
#model.save(sc, "target/tmp/myRandomForestClassificationModel")
#sameModel = RandomForestModel.load(sc, "target/tmp/myRandomForestClassificationModel")

metrics = MulticlassMetrics(labelsAndPredictions)
precision = metrics.precision(label=1)
recall = metrics.recall(label=1)
Accuracy = metrics.accuracy
print("RandomForest Accuracy = %s" % Accuracy)


Accuracy = 0.803185887309


In [88]:
#---------------------10NN-----------------------------------------------
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y,train_size=0.75,test_size=0.25)
neigh = KNeighborsClassifier(n_neighbors=20)
neigh=neigh.fit(X_train, y_train) 
neigh_predict = neigh.predict(X_test)
print('10NN accuracy =',accuracy_score(y_test, neigh_predict))

('10NN accuracy =', 0.78280000000000005)
